In [202]:
import requests, bs4, pandas as pd, numpy as np
import hashlib # use this to hash game key
import collections 

In [230]:
# Web scrapping 
# Try an example on MIT SIMULATOR, https://www.sportschatexperts.com/index/capperhistory/capper_id/68/
def get_html(url):
    html = requests.get(url)
    try:
        html.raise_for_status()
    except Exception as exc:
        print('There was a problem: %s' % exc)
    html = bs4.BeautifulSoup(html.text, "html5lib")
    return html

def get_next_url(html):
    # find the next button
    return html.find('a', string='Next')['href']

def merge_table(url, stop_page=-1, data=[]):
    cur_page = 0
    while True:
        cur_page += 1
        print("\tProcess %s ..." % url)
        html = get_html(url)
        table = html.find('section', class_='leaderboards diffpadding').div.table
        table_body = table.find('tbody')       
        rows = table_body.find_all('tr', class_='leaderboards_row2 fix-border-bottom ph-first-row')
        for row in rows:
            cols = row.find_all('td')
            # make sure the targeted row for extraction has 10 columns
            if len(cols) == 10:
                # only want to keep these columns
                keep_columns = [1, 2, 3, 4, 5, 6, 7, 8]
                cols = [ele.text.strip() for ele in [cols[i] for i in keep_columns]]
                data.append([ele for ele in cols if ele]) # Get rid of empty values
        url = get_next_url(html)
        if (url == '#') or cur_page >= stop_page:
            break

In [105]:
data=[]
merge_table('https://www.sportschatexperts.com/index/capperhistory/capper_id/68/', 2)

	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/68/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/68/page/2 ...


In [169]:
# Create a dataframe to hold historical bets for the handicapper 
# Add column headers

df = pd.DataFrame(data, columns=['League', 'Game', 'Date', 'Play', 'Line Selected', 'Type', 'Score', 'Result'])

In [170]:
df.head()

,League,Game,Date,Play,Line Selected,Type,Score,Result
0,NBA,New Orleans vs. LA Clippers,04/09/2018 22:30 EDT,Play on Over,Total: -105/+228½,Premium,113 - 100,Loss
1,MLB,Arizona vs. San Francisco,04/09/2018 22:15 EDT,Play on San Francisco,Money Line: +127,Premium,2 - 1,Loss
2,MLB,Tampa Bay vs. Chi White Sox,04/09/2018 14:10 EDT,Play on Under,Total: -110/+8½,Premium,5 - 4,Loss
3,MLB,NY Mets vs. Washington,04/08/2018 20:08 EDT,Play on Under,Total: -100/+8,Premium,6 - 5,Loss
4,NBA,Detroit vs. Memphis,04/08/2018 15:30 EDT,Play on Over,Total: -110/+202½,Premium,117 - 130,Win


In [171]:
# produce a hashed gamekey for the game
df["GameKey"] = df[['League', 'Game', 'Date']].apply(lambda x: hashlib.md5(''.join(x).encode('utf-8')).hexdigest(), axis=1)

# home team is always on the right?
df["Home"] = df["Game"].apply(lambda x: x.split('vs.')[1].strip().upper())

# away team is always on the left?
df["Away"] = df["Game"].apply(lambda x: x.split('vs.')[0].strip().upper())

# strip out type of bet
df["Bet"] = df["Line Selected"].apply(lambda x: x.split(':')[0].strip().upper())

# strip out which team the bet is on
df["On"] = df["Play"].apply(lambda x: x.split('Play on ')[1].strip().upper())

# convert game time to datetime format 
df["GameTime"] = df["Date"].apply(lambda x: pd.to_datetime(x))

# strip out juice
df["Juice"] = df["Line Selected"].apply(lambda x: x.split(': ')[1].strip() if ("Money Line" in x) else (x.split(': ')[1].split('/')[0].strip() if ("Total" in x) else x.split(': ')[1].split('/')[1].strip()))

# MLWinner: either home or away team for ML bets
conditions = [
    (df['On'] == df['Home']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Win'), 
    (df['On'] == df['Away']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Win'), 
    (df['On'] == df['Home']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Loss'), 
    (df['On'] == df['Away']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Loss')
]
choices = ['Home', 'Away', 'Away', 'Home']
df['MLWinner'] = np.select(conditions, choices, default='')

# MLBet: either home or away team for ML bets
conditions = [
    (df['On'] == df['Home']) & (df['Bet'] == 'MONEY LINE'), 
    (df['On'] == df['Away']) & (df['Bet'] == 'MONEY LINE')
]
choices = ['Home', 'Away']
df['MLBet'] = np.select(conditions, choices, default='')

In [195]:
columns_to_keep = ['League', 'GameKey', 'GameTime', 
                   'Home', 'Away',
                   'Bet', 'On',
                   'Juice', 'Type', 'Result',
                   'MLBet', 'MLWinner']
df[columns_to_keep].to_pickle('HC1.gz', compression="gzip")

In [197]:
mydf = pd.read_pickle('HC1.gz', compression='gzip')
mydf.head()

,League,GameKey,GameTime,Home,Away,Bet,On,Juice,Type,Result,MLBet,MLWinner
0,NBA,9a1a81685e0da902360fad2492a6ce4d,2018-04-09 22:30:00,LA CLIPPERS,NEW ORLEANS,TOTAL,OVER,-105,Premium,Loss,,
1,MLB,d6fe25a28380439cf630fb94d41017f9,2018-04-09 22:15:00,SAN FRANCISCO,ARIZONA,MONEY LINE,SAN FRANCISCO,+127,Premium,Loss,Home,Away
2,MLB,4b9d28b448297de1532898da287415a0,2018-04-09 14:10:00,CHI WHITE SOX,TAMPA BAY,TOTAL,UNDER,-110,Premium,Loss,,
3,MLB,526a94a26dd065c3cc6f13d21efccc24,2018-04-08 20:08:00,WASHINGTON,NY METS,TOTAL,UNDER,-100,Premium,Loss,,
4,NBA,2e64c0647d1f722981c283c2564acfcb,2018-04-08 15:30:00,MEMPHIS,DETROIT,TOTAL,OVER,-110,Premium,Win,,


In [239]:
def crawl_history():
    names = [
    'Smart',
    'Bartley',
    'Aronson',
    'Burns',
    'Barone', # not in Ji's original list
    'Bitler',
    'Power',
    'Ross',
    'Diamond',
    'Trapp',
    'Sports',
    'Eddie',
    'Schule',
    'DAmico',
    'Duffy',
    'Thomas',
    'Hunter',
    'Compeau', # not in Ji's original list
    'Syndicate',
    'Lundin',
    'Simulator',
    'Wilson',
    'Monohan',
    'Vinceletti',
    'Rickenbach',
    'Higgs',
    'Nover',
    'Brown',
    'Karpinski',
    'Rogers'
    ]

    links = [
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/44/", 
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/33/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/50/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/67/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/69/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/65/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/62/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/66/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/36/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/29/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/45/", 
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/38/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/52/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/35/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/37/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/53/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/41/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/56/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/58/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/61/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/68/", 
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/30/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/43/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/46/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/57/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/40/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/51/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/42/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/60/",
    "https://www.sportschatexperts.com/index/capperhistory/capper_id/55/",
    ]
    
    handicappers_dict = collections.OrderedDict(zip(names, links))
    
    for k, v in handicappers_dict.items():
        data=[]
        merge_table(v, 2, data)
        #print(data)
        df = pd.DataFrame(data, columns=['League', 'Game', 'Date', 'Play', 'Line Selected', 'Type', 'Score', 'Result'])
        filename = 'HC' + str(list(handicappers_dict.keys()).index(k)+1)
        
        # produce a hashed gamekey for the game
        df["GameKey"] = df[['League', 'Game', 'Date']].apply(lambda x: hashlib.md5(''.join(x).encode('utf-8')).hexdigest(), axis=1)

        # home team is always on the right?
        df["Home"] = df["Game"].apply(lambda x: x.split('vs.')[1].strip().upper())

        # away team is always on the left?
        df["Away"] = df["Game"].apply(lambda x: x.split('vs.')[0].strip().upper())

        # strip out type of bet
        df["Bet"] = df["Line Selected"].apply(lambda x: x.split(':')[0].strip().upper())

        # strip out which team the bet is on
        df["On"] = df["Play"].apply(lambda x: x.split('Play on ')[1].strip().upper())

        # convert game time to datetime format 
        df["GameTime"] = df["Date"].apply(lambda x: pd.to_datetime(x))

        # strip out juice
        df["Juice"] = df["Line Selected"].apply(lambda x: x.split(': ')[1].strip() if ("Money Line" in x) else (x.split(': ')[1].split('/')[0].strip() if ("Total" in x) else x.split(': ')[1].split('/')[1].strip()))

        # MLWinner: either home or away team for ML bets
        conditions = [
        (df['On'] == df['Home']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Win'), 
        (df['On'] == df['Away']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Win'), 
        (df['On'] == df['Home']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Loss'), 
        (df['On'] == df['Away']) & (df['Bet'] == 'MONEY LINE') & (df['Result'] == 'Loss')
        ]
        choices = ['Home', 'Away', 'Away', 'Home']
        df['MLWinner'] = np.select(conditions, choices, default='')

        # MLBet: either home or away team for ML bets
        conditions = [
        (df['On'] == df['Home']) & (df['Bet'] == 'MONEY LINE'), 
        (df['On'] == df['Away']) & (df['Bet'] == 'MONEY LINE')
        ]
        choices = ['Home', 'Away']
        df['MLBet'] = np.select(conditions, choices, default='')
        
        columns_to_keep = ['League', 'GameKey', 'GameTime', 
                   'Home', 'Away',
                   'Bet', 'On',
                   'Juice', 'Type', 'Result',
                   'MLBet', 'MLWinner']
        #print(df[columns_to_keep].head())
        df[columns_to_keep].to_pickle(filename + '.gz', compression="gzip")       

crawl_history()

	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/44/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/44/page/2 ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/33/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/33/page/2 ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/50/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/50/page/2 ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/67/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/67/page/2 ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/69/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/69/page/2 ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/65/ ...
	Process https://www.sportschatexperts.com/index/capperhistory/capper_id/65/pag

In [245]:
hc30 = pd.read_pickle('HC30.gz', compression='gzip')
hc30

,League,GameKey,GameTime,Home,Away,Bet,On,Juice,Type,Result,MLBet,MLWinner
0,MLB,3a4f8adc60ca77db09c8a65061322a2e,2018-04-11 13:10:00,MINNESOTA,HOUSTON,MONEY LINE,HOUSTON,-155,Premium,Loss,Away,Home
1,NBA,f87ad4c949d7f3c1e61011d57b2bd224,2018-04-10 21:00:00,UTAH,GOLDEN STATE,POINT SPREAD,GOLDEN STATE,-110,Premium,Loss,,
2,MLB,4593684777e736cfe1602293fb60c280,2018-04-10 20:05:00,TEXAS,LA ANGELS,MONEY LINE,LA ANGELS,-136,Premium,Win,Away,Away
3,NBA,2f803c260666b03ac895c70285a22af1,2018-04-10 19:30:00,ATLANTA,PHILADELPHIA,POINT SPREAD,ATLANTA,-110,Premium,Win,,
4,MLB,8cf116fbdfeff04ff1ad92eddda911c6,2018-04-10 14:10:00,CHI WHITE SOX,TAMPA BAY,TOTAL,OVER,-120,Premium,Win,,
5,NBA,83800cd0f41a558a59346745c4f49896,2018-04-09 21:00:00,DENVER,PORTLAND,MONEY LINE,DENVER,-180,Premium,Win,Home,Home
6,MLB,08c723b135840a9e9bb8af12760e11d1,2018-04-09 20:15:00,KANSAS CITY,SEATTLE,MONEY LINE,SEATTLE,-110,Premium,Loss,Away,Home
7,MLB,d80d283b4243018440b53b6c24d821ad,2018-04-09 18:10:00,CLEVELAND,DETROIT,TOTAL,OVER,-115,Premium,Loss,,
8,MLB,30260439014bdc2a6b9834b75d10bf91,2018-04-09 14:20:00,CHI CUBS,PITTSBURGH,MONEY LINE,CHI CUBS,-149,Premium,Push,Home,
9,MLB,526a94a26dd065c3cc6f13d21efccc24,2018-04-08 20:08:00,WASHINGTON,NY METS,TOTAL,UNDER,-113,Premium,Loss,,


In [244]:
hc15 = pd.read_pickle('HC15.gz', compression='gzip')
hc15

,League,GameKey,GameTime,Home,Away,Bet,On,Juice,Type,Result,MLBet,MLWinner
0,NBA,2f803c260666b03ac895c70285a22af1,2018-04-10 19:30:00,ATLANTA,PHILADELPHIA,POINT SPREAD,PHILADELPHIA,-110,Premium,Loss,,
1,NBA,767815408931259f1e14dea4b13110a1,2018-04-10 19:00:00,INDIANA,CHARLOTTE,POINT SPREAD,CHARLOTTE,-105,Premium,Win,,
2,NBA,767815408931259f1e14dea4b13110a1,2018-04-10 19:00:00,INDIANA,CHARLOTTE,TOTAL,OVER,-105,Premium,Loss,,
3,NBA,9a1a81685e0da902360fad2492a6ce4d,2018-04-09 22:30:00,LA CLIPPERS,NEW ORLEANS,POINT SPREAD,NEW ORLEANS,-105,Premium,Win,,
4,NBA,9a1a81685e0da902360fad2492a6ce4d,2018-04-09 22:30:00,LA CLIPPERS,NEW ORLEANS,TOTAL,OVER,+102,Premium,Loss,,
5,NBA,83800cd0f41a558a59346745c4f49896,2018-04-09 21:00:00,DENVER,PORTLAND,TOTAL,OVER,-110,Premium,Loss,,
6,NBA,2e64c0647d1f722981c283c2564acfcb,2018-04-08 15:30:00,MEMPHIS,DETROIT,POINT SPREAD,DETROIT,-110,Premium,Loss,,
7,MLB,9bc5fe20901f91e6d00f8331ee319643,2018-04-08 14:10:00,HOUSTON,SAN DIEGO,MONEY LINE,SAN DIEGO,+245,Premium,Loss,Away,Home
8,NBA,8f483e96e40ee117fd04aaeeb7f20aaf,2018-04-08 13:00:00,CHARLOTTE,INDIANA,POINT SPREAD,INDIANA,-102,Premium,Win,,
9,MLB,2430ae2e15a2944cac71c22ab2022e85,2018-04-07 21:07:00,LA ANGELS,OAKLAND,MONEY LINE,OAKLAND,-101,Premium,Win,Away,Away


In [ ]:
# additional features to make
"""
num_win_bets_specific_sport,
num_win_bets_all_sports,
num_loss_specific_sport,
num_loss_all_sports,
win_percentage_specific_sport,
win_percentage_all_sports,
avg_juice_on_bets_specific_sport, 
avg_juice_on_bets_all_sports,
current_win_streak_specific_sport, 
current_win_streak_all_sports,
current_loss_streak_specific_sport, 
current_loss_streak_all_sports,
longest_win_streak_specific_sport,
longest_win_streak_all_sports,
longest_loss_streak_specific_sport,
longest_loss_streak_all_sports
"""